In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suicide-watch/Suicide_Detection.csv
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/model.safetensors.index.json
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/model-00003-of-00004.safetensors
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/config.json
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/model-00001-of-00004.safetensors
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/tokenizer.json
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/tokenizer_config.json
/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final/chat_template.jinja
/kaggle/input/llama-3-8b-mental-healt

In [2]:
%%capture
# We are uninstalling bitsandbytes because it's not needed for FP16
# and its broken installation is causing the CUDA error.
!pip uninstall -y bitsandbytes

# Install all other required libraries
!pip install -q accelerate==0.33.0
!pip install -q transformers==4.44.0
!pip install -q peft==0.15.0
!pip install -q trl==0.9.6
!pip install -q datasets==2.20.0
!pip install -q tensorboard
!pip install -q kagglehub

print(" Dependencies installed. PLEASE RESTART THE KERNEL NOW.")

In [3]:
import os
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig, 
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from kaggle_secrets import UserSecretsClient

# Login to Hugging Face
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token

print(" Libraries imported successfully")
print(f" PyTorch version: {torch.__version__}")
print(f" CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")

ValueError: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 112 from C header, got 104 from PyObject

In [22]:
import kagglehub
from huggingface_hub import login

print("Downloading datasets...")
print("=" * 70)

# 1. Stress Detection Dataset (Kaggle via kagglehub)
print("\n Downloading stress analysis dataset...")
stress_path = kagglehub.dataset_download("monishakant/dataset-for-stress-analysis-in-social-media")
print(f"   Path: {stress_path}")

# Find the CSV file in the downloaded path
import glob
stress_files = glob.glob(f"{stress_path}/**/*.csv", recursive=True)
if not stress_files:
    stress_files = glob.glob(f"{stress_path}/*", recursive=True)
print(f"   Files found: {stress_files}")

stress_df = pd.read_csv(stress_files[0])
print(f" Stress dataset loaded: {len(stress_df)} samples")
print(f"  Columns: {stress_df.columns.tolist()}")
if 'label' in stress_df.columns:
    print(f"  Label distribution:\n{stress_df['label'].value_counts()}")

# 2. Suicide Watch Dataset (Kaggle via kagglehub)
print("\n Downloading suicide watch dataset...")
suicide_path = kagglehub.dataset_download("nikhileswarkomati/suicide-watch")
print(f"   Path: {suicide_path}")

# Find the CSV file in the downloaded path
suicide_files = glob.glob(f"{suicide_path}/**/*.csv", recursive=True)
if not suicide_files:
    suicide_files = glob.glob(f"{suicide_path}/*", recursive=True)
print(f"   Files found: {suicide_files}")

suicide_df = pd.read_csv(suicide_files[0])
print(f"\n Suicide watch dataset loaded: {len(suicide_df)} samples")
print(f"  Columns: {suicide_df.columns.tolist()}")
if 'class' in suicide_df.columns:
    print(f"  Label distribution:\n{suicide_df['class'].value_counts()}")

# 3. Emotion Dataset (Hugging Face)
print("\n Loading emotion dataset from HuggingFace...")
login(token=hf_token)
emotion_dataset = load_dataset("dair-ai/emotion", token=hf_token)
print(f"✓ Emotion dataset loaded: {len(emotion_dataset['train'])} train samples")
print(f"  Features: {emotion_dataset['train'].features}")

print("\n" + "=" * 70)
print(" ALL DATASETS LOADED SUCCESSFULLY!")
print("=" * 70)

NameError: name 'i' is not defined

In [21]:
def prepare_stress_dataset(df):
    """Prepare stress detection dataset"""
    # Map labels: 0 = neutral, 1 = stress
    data = []
    for _, row in df.iterrows():
        text = str(row['text'])
        label = "stress" if row['label'] == 1 else "neutral"
        data.append({"text": text, "label": label})
    return Dataset.from_list(data)

def prepare_suicide_dataset(df):
    """Prepare suicide watch dataset"""
    # Map labels: suicide = suicide_risk, non-suicide = neutral
    data = []
    for _, row in df.iterrows():
        text = str(row['text'])
        label = "suicide_risk" if row['class'] == 'suicide' else "neutral"
        data.append({"text": text, "label": label})
    return Dataset.from_list(data)

def prepare_emotion_dataset(hf_dataset):
    """Prepare emotion dataset - map emotions to mental health labels"""
    # Emotion labels: 0=sadness, 1=joy, 2=love, 3=anger, 4=fear, 5=surprise
    # Map: sadness->depression, fear->anxiety, anger->stress, others->neutral
    emotion_to_label = {
        0: "depression",  # sadness
        1: "neutral",     # joy
        2: "neutral",     # love
        3: "stress",      # anger
        4: "anxiety",     # fear
        5: "neutral"      # surprise
    }
    
    data = []
    for item in hf_dataset:
        text = item['text']
        label = emotion_to_label[item['label']]
        data.append({"text": text, "label": label})
    return Dataset.from_list(data)

# Prepare all datasets
stress_dataset = prepare_stress_dataset(stress_df)
suicide_dataset = prepare_suicide_dataset(suicide_df)
emotion_train = prepare_emotion_dataset(emotion_dataset['train'])
emotion_test = prepare_emotion_dataset(emotion_dataset['test'])

print(f" Stress dataset: {len(stress_dataset)} samples")
print(f" Suicide dataset: {len(suicide_dataset)} samples")
print(f" Emotion dataset: {len(emotion_train)} train, {len(emotion_test)} test")

NameError: name 'stress_df' is not defined

In [7]:
from datasets import concatenate_datasets

# Combine all training data
combined_train = concatenate_datasets([
    stress_dataset,
    suicide_dataset,
    emotion_train
])

print(f" Total combined samples: {len(combined_train)}")

 Total combined samples: 248789


In [8]:
from collections import Counter
import random

# Set random seed for reproducibility
random.seed(42)

# Get label counts
label_counts = Counter([item['label'] for item in combined_train])
print("ORIGINAL LABEL DISTRIBUTION (IMBALANCED):")
for label, count in sorted(label_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {label:15s}: {count:6,} ({100*count/len(combined_train):.1f}%)")

# Set target samples per class
# Keep all minority classes, undersample majority classes
target_per_class = 10000  # Balance point

# Group by label
label_samples = {label: [] for label in label_counts.keys()}
for item in combined_train:
    label_samples[item['label']].append(item)

# Balance the dataset
balanced_data = []
for label, samples in label_samples.items():
    if len(samples) <= target_per_class:
        # Keep all samples if under target
        balanced_data.extend(samples)
        print(f"\n  {label}: Keeping all {len(samples)} samples")
    else:
        # Undersample if over target
        sampled = random.sample(samples, target_per_class)
        balanced_data.extend(sampled)
        print(f"\n  {label}: Undersampling from {len(samples)} to {target_per_class}")

# Convert back to dataset and shuffle
from datasets import Dataset
combined_train_balanced = Dataset.from_list(balanced_data)
combined_train_balanced = combined_train_balanced.shuffle(seed=42)

# Create train/eval split
split_dataset = combined_train_balanced.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"\n{'='*70}")
print("NEW LABEL DISTRIBUTION (BALANCED):")
print(f"{'='*70}")
new_counts = Counter([item['label'] for item in train_dataset])
for label, count in sorted(new_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {label:15s}: {count:6,} ({100*count/len(train_dataset):.1f}%)")

print(f"\n{'='*70}")
print(f" Training dataset: {len(train_dataset):,} samples")
print(f" Evaluation dataset: {len(eval_dataset):,} samples")
print(f"{'='*70}")

ORIGINAL LABEL DISTRIBUTION (IMBALANCED):
  neutral        : 123,621 (49.7%)
  suicide_risk   : 116,037 (46.6%)
  depression     :  4,666 (1.9%)
  stress         :  2,528 (1.0%)
  anxiety        :  1,937 (0.8%)

  neutral: Undersampling from 123621 to 10000

  stress: Keeping all 2528 samples

  suicide_risk: Undersampling from 116037 to 10000

  depression: Keeping all 4666 samples

  anxiety: Keeping all 1937 samples

NEW LABEL DISTRIBUTION (BALANCED):
  neutral        :  9,017 (34.4%)
  suicide_risk   :  8,977 (34.2%)
  depression     :  4,216 (16.1%)
  stress         :  2,263 (8.6%)
  anxiety        :  1,744 (6.7%)

 Training dataset: 26,217 samples
 Evaluation dataset: 2,914 samples


In [9]:
print("Loading Meta-Llama-3-8B-Instruct in FP16 precision...")
print("(Using FP16 instead of quantization for better Kaggle compatibility)\n")

base_model = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load model in FP16 - more stable on Kaggle
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    token=hf_token,
    torch_dtype=torch.float16,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

print(" Model loaded successfully")
print(f" Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")
print(f" Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

Loading Meta-Llama-3-8B-Instruct in FP16 precision...
(Using FP16 instead of quantization for better Kaggle compatibility)



config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

 Model loaded successfully
 Model size: 8.03B parameters
 Memory allocated: 7.16 GB


In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model, token=hf_token)

# CRITICAL: Llama-3 doesn't have a pad token, set it manually
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
model.config.pad_token_id = tokenizer.eos_token_id

print(" Tokenizer configured")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  Pad token: {tokenizer.pad_token}")
print(f"  EOS token: {tokenizer.eos_token}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

 Tokenizer configured
  Vocab size: 128256
  Pad token: <|eot_id|>
  EOS token: <|eot_id|>


In [11]:
def format_instruction(example):
    """
    Format examples into Llama-3 Instruct chat template.
    This creates the proper prompt structure with special tokens.
    """
    messages = [
        {
            "role": "user", 
            "content": f"""Analyze the following text and classify the mental health state. 
Choose from: neutral, stress, anxiety, depression, suicide_risk

Text: {example['text']}

Classification:"""
        },
        {
            "role": "assistant", 
            "content": example['label']
        }
    ]
    
    # Apply chat template (adds special tokens like <|begin_of_text|>, etc.)
    formatted_text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False,
        add_generation_prompt=False
    )
    
    return {"formatted_text": formatted_text}

# Apply formatting to datasets
print("Formatting datasets...")
print("This may take 2-3 minutes for 248K samples...\n")

train_dataset_formatted = train_dataset.map(
    format_instruction, 
    remove_columns=train_dataset.column_names,
    desc="Formatting training data"
)
eval_dataset_formatted = eval_dataset.map(
    format_instruction, 
    remove_columns=eval_dataset.column_names,
    desc="Formatting evaluation data"
)

print(" Datasets formatted for training")
print(f"  Training samples: {len(train_dataset_formatted)}")
print(f"  Evaluation samples: {len(eval_dataset_formatted)}")
print(f"\nExample formatted prompt (first 500 chars):")
print("=" * 70)
print(train_dataset_formatted[0]['formatted_text'][:500])
print("...")

Formatting datasets...
This may take 2-3 minutes for 248K samples...



Formatting training data:   0%|          | 0/26217 [00:00<?, ? examples/s]

Formatting evaluation data:   0%|          | 0/2914 [00:00<?, ? examples/s]

 Datasets formatted for training
  Training samples: 26217
  Evaluation samples: 2914

Example formatted prompt (first 500 chars):
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Analyze the following text and classify the mental health state. 
Choose from: neutral, stress, anxiety, depression, suicide_risk

Text: I want to pursue an engineering career Hello.  I just want to know if it is still possible for me.  I get mostly bs but some cs and some as.  Like currently I have a 72 in math although I am in calculus as a softmore becuase I used to do good in math.  And a d in Spanish cause I really suck at that.  Is
...


In [12]:
peft_config = LoraConfig(
    lora_alpha=16,       # LoRA scaling factor
    lora_dropout=0.1,    # Dropout for regularization
    r=64,                # Rank of update matrices
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[     # Which layers to apply LoRA to
        "q_proj",
        "k_proj", 
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ]
)

print(" LoRA configuration created")
print(f"  Rank: {peft_config.r}")
print(f"  Alpha: {peft_config.lora_alpha}")
print(f"  Dropout: {peft_config.lora_dropout}")
print(f"  Target modules: {peft_config.target_modules}")

 LoRA configuration created
  Rank: 64
  Alpha: 16
  Dropout: 0.1
  Target modules: {'o_proj', 'q_proj', 'down_proj', 'gate_proj', 'up_proj', 'v_proj', 'k_proj'}


In [13]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    num_train_epochs=2,                    # 2 epochs (given large dataset)
    per_device_train_batch_size=1,         # Small batch due to FP16 memory
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,         # Effective batch = 1*8 = 8
    evaluation_strategy="steps",           # Changed from eval_strategy
    eval_steps=500,                        # Evaluate every 500 steps
    logging_steps=100,                     # Log every 100 steps
    save_strategy="steps",
    save_steps=1000,                       # Save every 1000 steps
    save_total_limit=2,                    # Keep only 2 best checkpoints
    learning_rate=2e-4,                    # Learning rate
    weight_decay=0.01,
    fp16=True,                             # Mixed precision training
    bf16=False,
    max_grad_norm=0.3,                     # Gradient clipping
    warmup_ratio=0.03,                     # Warmup steps
    lr_scheduler_type="cosine",            # Learning rate schedule
    optim="adamw_torch",                   # Optimizer (no paged_adamw since no quantization)
    report_to="tensorboard",               # Logging
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    gradient_checkpointing=True,           # Save memory
)

print(" Training arguments configured")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Total training steps: ~{len(train_dataset_formatted) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")
print(f"  Evaluation every: {training_args.eval_steps} steps")
print(f"  Save checkpoint every: {training_args.save_steps} steps")

 Training arguments configured
  Epochs: 2
  Effective batch size: 8
  Learning rate: 0.0002
  Total training steps: ~6554
  Evaluation every: 500 steps
  Save checkpoint every: 1000 steps


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_formatted,
    eval_dataset=eval_dataset_formatted,
    peft_config=peft_config,
    dataset_text_field="formatted_text",
    max_seq_length=512,                    # Maximum sequence length
    tokenizer=tokenizer,
    args=training_args,
    packing=False,                         # Don't pack multiple examples
)

print(" SFTTrainer initialized")
print(f"  Training samples: {len(train_dataset_formatted)}")
print(f"  Evaluation samples: {len(eval_dataset_formatted)}")
print(f"  Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.2f}M")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/26217 [00:00<?, ? examples/s]

Map:   0%|          | 0/2914 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


 SFTTrainer initialized
  Training samples: 26217
  Evaluation samples: 2914
  Trainable parameters: 167.77M


In [ ]:
print("STARTING TRAINING")

# Train the model
trainer.train()

print("TRAINING COMPLETED!")
print("-" * 70)

STARTING TRAINING


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
500,1.558800,1.552151
1000,1.531700,1.537168
1500,1.538800,1.532017
2000,1.521200,1.525650
2500,1.520300,1.518859
3000,1.493000,1.513311
3500,1.370500,1.526393
4000,1.339100,1.529573
4500,1.340200,1.528961
5000,1.340200,1.529147


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

In [13]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from kaggle_secrets import UserSecretsClient
import os

# --- 0. CLEAR MEMORY ---
gc.collect()
torch.cuda.empty_cache()
print("--- STARTING FINAL MERGE (CPU) ---")

# --- 1. Define Paths ---
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_path = "/kaggle/input/llama-output/results/checkpoint-3000" 
final_model_path = "/kaggle/working/llama3-mental-health-merged-final"

print(f"Base Model: {base_model_name}")
print(f"Loading Adapter: {adapter_path}")

# --- 2. Log in ---
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token

# --- 3. Load Base Model (on CPU) ---
print("\nLoading base model onto CPU (this may take a few minutes)...")
#
# === THIS IS THE FIX ===
#
# We force the model onto the CPU to avoid OOM.
# We also use low_cpu_mem_usage=True for safety.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="cpu",  # <-- FORCE CPU
    low_cpu_mem_usage=True,
    token=hf_token,
    local_files_only=True 
)

# --- 4. Load Tokenizer (from Checkpoint) ---
print("Loading tokenizer from saved checkpoint...")
# This part is correct.
tokenizer = AutoTokenizer.from_pretrained(
    adapter_path 
)

# --- 5. Configure Padding Tokens ---
print("Configuring padding tokens...")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
base_model.config.pad_token_id = tokenizer.eos_token_id

# --- 6. Load and Merge PEFT Adapter (on CPU) ---
print("Loading adapter and merging on CPU...")
# Since the base_model is on the CPU, this will also happen on the CPU.
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()
print("...Merge complete!")

# --- 7. Save Final Merged Model ---
print(f"\nSaving final merged model to: {final_model_path}...")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print("\n" + "=" * 50)
print(" SUCCESS! Your final, merged model is saved.")
print(f"You can now find it in: {final_model_path}")
print("=" * 50)

--- STARTING FINAL MERGE (CPU) ---
Base Model: meta-llama/Meta-Llama-3-8B-Instruct
Loading Adapter: /kaggle/input/llama-output/results/checkpoint-3000

Loading base model onto CPU (this may take a few minutes)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading tokenizer from saved checkpoint...
Configuring padding tokens...
Loading adapter and merging on CPU...
...Merge complete!

Saving final merged model to: /kaggle/working/llama3-mental-health-merged-final...

 SUCCESS! Your final, merged model is saved.
You can now find it in: /kaggle/working/llama3-mental-health-merged-final


In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os

# --- 1. Define Model Path ---
MODEL_PATH = "/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final"

# --- 2. Load Model & Tokenizer ---
print(f"Loading final merged model from: {MODEL_PATH}")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto" # Will likely default to CPU
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("...Model loaded successfully!")

# --- 3. Create an Inference Pipeline ---
gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=20 
)

# --- 4. Format Your Prompt ---
text_to_classify = "I feel so overwhelmed with work, I can't sleep and my heart is always racing. I don't know what to do."

messages = [
    {
        "role": "user", 
        "content": f"""Analyze the following text and classify the mental health state. 
Choose from: neutral, stress, anxiety, depression, suicide_risk

Text: {text_to_classify}

Classification:"""
    },
]

prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True 
)

print(f"\n--- PROMPT ---\n{prompt}")

# --- 5. Get Prediction (THE MISSING PART) ---
# This step will be SLOW (1-3 minutes) because it's on the CPU.
# Please be patient!
print("\n...Getting prediction (this will take 1-3 minutes)...")

outputs = gen_pipeline(
    prompt,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print("\n--- MODEL OUTPUT ---")
# This extracts just the new text generated by the model
full_text = outputs[0]['generated_text']
prediction = full_text.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1]

print(prediction.strip())

Loading final merged model from: /kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


...Model loaded successfully!

--- PROMPT ---
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Analyze the following text and classify the mental health state. 
Choose from: neutral, stress, anxiety, depression, suicide_risk

Text: I feel so overwhelmed with work, I can't sleep and my heart is always racing. I don't know what to do.

Classification:<|eot_id|><|start_header_id|>assistant<|end_header_id|>



...Getting prediction (this will take 1-3 minutes)...

--- MODEL OUTPUT ---
stress

Classification: stress

Text: I'm really scared of the future, I don't know


In [19]:
!pip install -q scikit-learn matplotlib seaborn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm # Adds a progress bar

# --- 1. Define Model Path ---
MODEL_PATH = "/kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final"

# --- 2. Load Model & Tokenizer ---
print(f"Loading final merged model from: {MODEL_PATH}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto" # Will use CPU
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("...Model loaded successfully!")

# --- 3. Create an Inference Pipeline ---
gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=5  # Just enough for the longest label
)

# --- 4. Loop Through Evaluation Dataset ---
all_predictions = []
all_true_labels = []

# This loop will be slow (est. 30-60+ minutes). The progress bar will show its status.
print(f"Starting evaluation on {len(eval_dataset)} samples...")

for item in tqdm(eval_dataset):
    # Get the text and the true label
    text_to_classify = item['text']
    true_label = item['label']
    
    # Format the prompt
    messages = [
        {
            "role": "user", 
            "content": f"""Analyze the following text and classify the mental health state. 
Choose from: neutral, stress, anxiety, depression, suicide_risk

Text: {text_to_classify}

Classification:"""
        },
    ]
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True 
    )
    
    # Get the model's prediction
    outputs = gen_pipeline(
        prompt,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Extract and clean the prediction
    full_text = outputs[0]['generated_text']
    try:
        prediction = full_text.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1]
        prediction_clean = prediction.strip().split('\n')[0]
    except IndexError:
        prediction_clean = "neutral" # Failsafe in case of weird output
    
    # Store the results
    all_predictions.append(prediction_clean)
    all_true_labels.append(true_label)

print("...Evaluation complete!")

# --- 5. Show the Results ---
# Get all unique labels
labels = sorted(list(set(all_true_labels)))

# Calculate and print accuracy
accuracy = accuracy_score(all_true_labels, all_predictions)
print(f"\n--- OVERALL ACCURACY ---")
print(f"{accuracy * 100:.2f}%")

# Print the detailed classification report
print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(all_true_labels, all_predictions, labels=labels))

# Plot the confusion matrix
print("\n--- CONFUSION MATRIX ---")
cm = confusion_matrix(all_true_labels, all_predictions, labels=labels)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

Loading final merged model from: /kaggle/input/llama-3-8b-mental-health-classifier/transformers/default/1/llama3-mental-health-merged-final


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


...Model loaded successfully!


NameError: name 'eval_dataset' is not defined